This is the latest notebook. There we will use additional information about purchases and prices of other currencies.

# Context

Initially, we had several million lines of raw data on 170+ currencies.

At the very beginning, we divided the data into tables for each currency. Next, we reduced the size and unified the format of table storage using quantization. 

After that we started to analyze only target currency prices, learned how to automatically build several dozens of the most useful features (using statistical tests and feature importance selection).

The **first** approach that comes to mind is to generate all of these attributes for all of the currencies, combine it into one table, and repeat the procedure of selecting the most useful features. By doing this, we will get the most informative dependencies between the target currency and the other. But this approach has a **disadvantage** - we will have about $170 \times 30 \approx 6K$ attributes for model training. This is expensive. In addition, it is hard to regulate the non-correlativity of the obtained attributes (this is very important in `feature_importance` selection).

# Solution

To combat this problem, we will initially select only those currencies that are **highly correlated** with the target currency. 

Note that the usual distances between vectors, which are obtained by projecting the values of the time series onto the time axis, will **not suit us**. For more details on the reasons for this, see **DOCS**. 

We will do the following - from each time series we will construct a new time series, the values of which at each moment of time are % changes of the values of the original ones. After that we will compare the distances between the obtained series. This approach has several advantages (you can also read about it in **DOCS**).

In [1]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import extraction_utils
import time
import preprocessing_utils

In [2]:
names = np.array([
    '1000LUNC_USDT_PERP', '1000SHIB_USDT_PERP', '1000XEC_USDT_PERP',
    '1INCH_USDT_PERP', 'AAVE_USDT_PERP', 'ADA_BUSD_PERP', 'ADA_USDT_PERP',
    'ALGO_USDT_PERP', 'ALICE_USDT_PERP', 'ALPHA_USDT_PERP', 'ANC_BUSD_PERP',
    'ANKR_USDT_PERP', 'ANT_USDT_PERP', 'APE_BUSD_PERP', 'APE_USDT_PERP',
    'API3_USDT_PERP', 'APT_USDT_PERP', 'ARPA_USDT_PERP', 'AR_USDT_PERP',
    'ATA_USDT_PERP', 'ATOM_USDT_PERP', 'AUDIO_USDT_PERP', 'AVAX_BUSD_PERP',
    'AVAX_USDT_PERP', 'AXS_USDT_PERP', 'BAKE_USDT_PERP', 'BAL_USDT_PERP',
    'BAND_USDT_PERP', 'BAT_USDT_PERP', 'BCH_USDT_PERP', 'BEL_USDT_PERP',
    'BLZ_USDT_PERP', 'BNB_BUSD_PERP', 'BNB_USDT_PERP', 'BNX_USDT_PERP',
    'BTCDOM_USDT_PERP', 'BTC_BUSD_PERP', 'BTC_USDT_CQ', 'BTC_USDT_PERP',
    'C98_USDT_PERP', 'CELO_USDT_PERP', 'CELR_USDT_PERP', 'CHR_USDT_PERP',
    'CHZ_USDT_PERP', 'COMP_USDT_PERP', 'COTI_USDT_PERP', 'CRV_USDT_PERP',
    'CTK_USDT_PERP', 'CTSI_USDT_PERP', 'CVC_USDT_PERP', 'CVX_USDT_PERP',
    'DAR_USDT_PERP', 'DASH_USDT_PERP', 'DEFI_USDT_PERP', 'DENT_USDT_PERP',
    'DGB_USDT_PERP', 'DODO_BUSD_PERP', 'DOGE_BUSD_PERP', 'DOGE_USDT_PERP',
    'DOT_BUSD_PERP', 'DOT_USDT_PERP', 'DUSK_USDT_PERP', 'DYDX_USDT_PERP',
    'EGLD_USDT_PERP', 'ENJ_USDT_PERP', 'ENS_USDT_PERP', 'EOS_USDT_PERP',
    'ETC_BUSD_PERP', 'ETC_USDT_PERP', 'ETH_BUSD_PERP', 'ETH_USDT_CQ',
    'ETH_USDT_PERP', 'FIL_BUSD_PERP', 'FIL_USDT_PERP', 'FLM_USDT_PERP',
    'FLOW_USDT_PERP', 'FOOTBALL_USDT_PERP', 'FTM_BUSD_PERP', 'FTM_USDT_PERP',
    'GALA_BUSD_PERP', 'GALA_USDT_PERP', 'GAL_BUSD_PERP', 'GAL_USDT_PERP',
    'GMT_BUSD_PERP', 'GMT_USDT_PERP', 'GRT_USDT_PERP', 'GTC_USDT_PERP',
    'HBAR_USDT_PERP', 'HNT_USDT_PERP', 'HOT_USDT_PERP', 'ICP_BUSD_PERP',
    'ICP_USDT_PERP', 'ICX_USDT_PERP', 'IMX_USDT_PERP', 'INJ_USDT_PERP',
    'IOST_USDT_PERP', 'IOTA_USDT_PERP', 'IOTX_USDT_PERP', 'JASMY_USDT_PERP',
    'KAVA_USDT_PERP', 'KLAY_USDT_PERP', 'KNC_USDT_PERP', 'KSM_USDT_PERP',
    'LDO_USDT_PERP', 'LINA_USDT_PERP', 'LINK_BUSD_PERP', 'LINK_USDT_PERP',
    'LIT_USDT_PERP', 'LPT_USDT_PERP', 'LRC_USDT_PERP', 'LTC_USDT_PERP',
    'LUNA2_USDT_PERP', 'MANA_USDT_PERP', 'MASK_USDT_PERP', 'MATIC_BUSD_PERP',
    'MATIC_USDT_PERP', 'MKR_USDT_PERP', 'MTL_USDT_PERP', 'NEAR_BUSD_PERP',
    'NEAR_USDT_PERP', 'NEO_USDT_PERP', 'NKN_USDT_PERP', 'OCEAN_USDT_PERP',
    'OGN_USDT_PERP', 'OMG_USDT_PERP', 'ONE_USDT_PERP', 'ONT_USDT_PERP',
    'OP_USDT_PERP', 'PEOPLE_USDT_PERP', 'QNT_USDT_PERP', 'QTUM_USDT_PERP',
    'RAY_USDT_PERP', 'REEF_USDT_PERP', 'REN_USDT_PERP', 'RLC_USDT_PERP',
    'ROSE_USDT_PERP', 'RSR_USDT_PERP', 'RUNE_USDT_PERP', 'RVN_USDT_PERP',
    'SAND_USDT_PERP', 'SFP_USDT_PERP', 'SKL_USDT_PERP', 'SNX_USDT_PERP',
    'SOL_BUSD_PERP', 'SOL_USDT_PERP', 'SPELL_USDT_PERP', 'SRM_USDT_PERP',
    'STG_USDT_PERP', 'STMX_USDT_PERP', 'STORJ_USDT_PERP', 'SUSHI_USDT_PERP',
    'SXP_USDT_PERP', 'THETA_USDT_PERP', 'TOMO_USDT_PERP', 'TRB_USDT_PERP',
    'TRX_BUSD_PERP', 'TRX_USDT_PERP', 'UNFI_USDT_PERP', 'UNI_BUSD_PERP',
    'UNI_USDT_PERP', 'VET_USDT_PERP', 'WAVES_BUSD_PERP', 'WAVES_USDT_PERP',
    'WOO_USDT_PERP', 'XEM_USDT_PERP', 'XLM_USDT_PERP', 'XMR_USDT_PERP',
    'XRP_BUSD_PERP', 'XRP_USDT_PERP', 'XTZ_USDT_PERP', 'YFI_USDT_PERP',
    'ZEC_USDT_PERP', 'ZEN_USDT_PERP', 'ZIL_USDT_PERP', 'ZRX_USDT_PERP'
],
                 dtype=object)

TARGET_NAME = 'CHZ_USDT_PERP_MIDPRICE'

In [3]:
# this dictionary was obtained in the one of runs previous notebook
fc_parameters = {'price_mean': {'mean_second_derivative_central': None,
  'augmented_dickey_fuller': [{'attr': 'pvalue', 'autolag': 'AIC'}],
  'partial_autocorrelation': [{'lag': 3}],
  'fft_coefficient': [{'attr': 'imag', 'coeff': 70}],
  'cwt_coefficients': [{'coeff': 0, 'w': 2, 'widths': (2, 5, 10, 20)}],
  'ar_coefficient': [{'coeff': 8, 'k': 10}],
  'skewness': None,
  'binned_entropy': [{'max_bins': 10}],
  'spkt_welch_density': [{'coeff': 2}],
  'friedrich_coefficients': [{'coeff': 3, 'm': 3, 'r': 30}],
  'autocorrelation': [{'lag': 3}],
  'permutation_entropy': [{'dimension': 3, 'tau': 1}],
  'mean_abs_change': None,
  'fft_aggregated': [{'aggtype': 'variance'}],
  'max_langevin_fixed_point': [{'m': 3, 'r': 30}],
  'kurtosis': None,
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0}],
  'mean_change': None,
  'time_reversal_asymmetry_statistic': [{'lag': 3}],
  'linear_trend': [{'attr': 'slope'}],
  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40}],
  'cid_ce': [{'normalize': True}, {'normalize': False}],
  'agg_linear_trend': [{'attr': 'slope', 'chunk_len': 50, 'f_agg': 'max'}],
  'root_mean_square': None,
  'variation_coefficient': None,
  'count_above_mean': None,
  'longest_strike_below_mean': None,
  'longest_strike_above_mean': None,
  'last_location_of_maximum': None}}

In [4]:
def corr(df1: pd.DataFrame, df2: pd.DataFrame) -> float:
    x = 100 * (df1['price_mean'].shift(-1) -
               df1['price_mean']) / df1['price_mean']
    y = 100 * (df2['price_mean'].shift(-1) -
               df2['price_mean']) / df2['price_mean']
    
    x_diff = x - np.mean(x)
    y_diff = y - np.mean(y)               
    numerator = np.sum(x_diff * y_diff)
    denominator = np.sqrt(np.sum(x_diff ** 2) * np.sum(y_diff ** 2))
    return numerator / denominator

In [5]:
quantized_df_dict = preprocessing_utils.load_tables(names=list(names) +
                                              [TARGET_NAME],
                                              path_from='../data/quantized')

In [6]:
target_table = quantized_df_dict[TARGET_NAME]

In [7]:
correlations = [(corr(quantized_df_dict[name], target_table), name)
                for name in names]

In [8]:
top_currencies = [corr_info[1] for corr_info in sorted(correlations)[-10:]]

In [9]:
print(f"We see that the most correlated currencies are the following 10:")
for curr in top_currencies:
    print("   ", curr)

We see that the most correlated currencies are the following 10:
    BNB_USDT_PERP
    ETC_USDT_PERP
    DOGE_USDT_PERP
    SOL_USDT_PERP
    BTC_BUSD_PERP
    MATIC_USDT_PERP
    ETH_BUSD_PERP
    ETH_USDT_PERP
    BTC_USDT_PERP
    CHZ_USDT_PERP


In [10]:
for curr_info in sorted(correlations)[-10:]:
    print(
        f"Correlation between '{curr_info[1]}' and '{TARGET_NAME}' is {curr_info[0]:.2f}"
    )

Correlation between 'BNB_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.21
Correlation between 'ETC_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.22
Correlation between 'DOGE_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.22
Correlation between 'SOL_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.22
Correlation between 'BTC_BUSD_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.23
Correlation between 'MATIC_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.24
Correlation between 'ETH_BUSD_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.24
Correlation between 'ETH_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.30
Correlation between 'BTC_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.32
Correlation between 'CHZ_USDT_PERP' and 'CHZ_USDT_PERP_MIDPRICE' is 0.82


In [11]:
# let's leave only the tables we need
del quantized_df_dict
top_currencies_dict = preprocessing_utils.load_tables(names=list(top_currencies) +
                                                [TARGET_NAME],
                                                path_from='../data/quantized')

In [12]:
# add information about which currency this `price_mean` is taken from
list_top_currencies = []
for i, name in enumerate(top_currencies):
    df = top_currencies_dict[name]
    cols = df.columns
    df.columns = ['(' + name + ') ' + col for col in cols]
    list_top_currencies.append(df)

target_table.columns = [
    '(' + TARGET_NAME + ') ' + col for col in target_table.columns
]

In [13]:
big_df = pd.concat([target_table]+list_top_currencies, axis=1).dropna()
index = big_df.index

In [14]:
len(index)

11106

Due to the fact that we just delete the rows with `NaN`, we have quite a lot of decreased dimensionality.\
The more correlated currencies we take, the fewer rows will remain; that is why we chose the top 10.

Now for each of the currencies we will create a series of dataframes for block cross validation with the count of the most important features (which were obtained in the previous `extract_features` notebook).\
After that, we will compare what kind of increment these features give, counted on correlated currencies.\
And at the very end let's try to select the most important ones among all (among the important features counted for all currencies).

In [15]:
target_table.head(3)

,(CHZ_USDT_PERP_MIDPRICE) price_mean
event_time,
2022-11-15 00:00:00.000,0.198392
2022-11-15 00:00:00.300,0.198341
2022-11-15 00:00:00.600,NaN


In [16]:
%%time
bcv_blocks_dict = {}
all_names = top_currencies + [TARGET_NAME]
all_dfs = list_top_currencies + [target_table]
for i, df in enumerate(all_dfs):
    bcv_blocks_dict[all_names[i]] = extraction_utils.bcv_extract_features(
        df=df.loc[index],
        n_blocks=5,
        target_col=f'({all_names[i]}) price_mean',
        n_windows=1500,
        window_size=200,
        lags=[1],
        n_jobs=8,
        mode='parallel',
        fc_parameters=fc_parameters['price_mean'])

current block: 1/5
========--------------------------------


Feature Extraction: 100%|██████████| 40/40 [00:05<00:00,  7.96it/s]


current block: 2/5
================------------------------


Feature Extraction: 100%|██████████| 40/40 [00:04<00:00,  8.32it/s]

current block: 3/5
========================----------------


...

current block: 4/5
================================--------


Feature Extraction: 100%|██████████| 40/40 [00:08<00:00,  4.93it/s]


current block: 5/5


Feature Extraction: 100%|██████████| 40/40 [00:08<00:00,  4.84it/s]

CPU times: user 1min 31s, sys: 15.9 s, total: 1min 47s
Wall time: 9min 52s


In [18]:
# remove common columns (time features) and data from the future (target col)
need_to_delete = ['hour', 'min', 'sec', 'ms', 'target']
for name in top_currencies:
    for i in range(5): # 5 = n_blocks
        for col in need_to_delete:
            if col in bcv_blocks_dict[name][i]:
                bcv_blocks_dict[name][i] = bcv_blocks_dict[name][i].drop([col], axis=1)
        bcv_blocks_dict[name][i].rename(columns={'price_lag1': f'({name} )price_mean_lag1'}, inplace=True)       

In [19]:
import selection_utils
from typing import List
import xgboost
from xgboost import XGBRegressor

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [20]:
def get_rmse(
        models: List[xgboost.sklearn.XGBRegressor],
        test_list: List[pd.DataFrame]
) -> List[List[float]]:
    """
    Predictions are made for each model in the `models` and the corresponding
    training dataset. Then rmse are calculated for all lengths of the
    prediction horizon from 1 to `len(test_list)`. That is, we look at the
    rmse of the predictions for the next `300ms`, the next `600ms`, and so on.  
    
    :param models: list of trained XGBRegressors
    :param test_list: test data list
    :return: list containing the list with rmse for all prediction horizons
    """
    assert len(models) == len(test_list), f'len(models)={len(models)}!={len(test_list)}=len(test_list)!'

    length = test_list[0].shape[0]

    losses = []
    for i, model in enumerate(models):
        losses.append([])
        x, y = test_list[i].drop(['target'], axis=1), test_list[i].loc[:, 'target']
        mse = .0
        for l in range(length):
            y_hat = model.predict(x.iloc[l:l + 1])
            mse += (y_hat - y.iloc[l]) ** 2
            losses[-1].append(np.sqrt(mse / (l + 1)))

    return losses

In [ ]:
%%time
# only target table available
test_size = 20
n_jobs = 8
train, test = [], []
for i in range(5):
    train.append(bcv_blocks_dict[TARGET_NAME][i].iloc[:-test_size])
    test.append(bcv_blocks_dict[TARGET_NAME][i].iloc[-test_size:])
    
start_time = time.time()   
models = selection_utils.get_fitted_models(train, n_jobs)
only_target_time = time.time() - start_time

only_target_rmse = get_rmse(models, test)

In [ ]:
%%time
# all other tables are available
test_size = 20
train, test = [], []
for i in range(5):
    train.append(pd.concat([bcv_blocks_dict[name][i].iloc[:-test_size] for name in [TARGET_NAME]+top_currencies], axis=1))
    test.append(pd.concat([bcv_blocks_dict[name][i].iloc[-test_size:] for name in [TARGET_NAME]+top_currencies], axis=1))
    
start_time = time.time()
models = selection_utils.get_fitted_models(train, n_jobs)
with_other_tables_time = time.time() - start_time

with_other_tables_rmse = get_rmse(models, test)

In [23]:
len(train[0].columns)

449

In [25]:
# we also can select most important features
importances = selection_utils.get_importance(models, train, mode='all')
selected_features = [el[0] for el in selection_utils.importance_select_features(importances, 0.8)][:35]

In [26]:
len(selected_features)

35

In [ ]:
%%time
# with selected features from other tables

test_size = 20
train, test = [], []
for i in range(5):
    train.append(pd.concat([bcv_blocks_dict[name][i].iloc[:-test_size] for name in [TARGET_NAME]+top_currencies], axis=1))
    test.append(pd.concat([bcv_blocks_dict[name][i].iloc[-test_size:] for name in [TARGET_NAME]+top_currencies], axis=1))

train = [df[selected_features + ['target']] for df in train]
test = [df[selected_features + ['target']] for df in test]

start_time = time.time()
models = selection_utils.get_fitted_models(train, n_jobs)
with_selected_features_time = time.time() - start_time

with_selected_features_rmse = get_rmse(models, test)

### Let's calculate rmse and summarize

In [28]:
# average of all models
avg_only_target_rmse = np.hstack(only_target_rmse).mean(axis=1)
avg_with_other_tables_rmse = np.hstack(with_other_tables_rmse).mean(axis=1)
avg_with_selected_features_rmse = np.hstack(with_selected_features_rmse).mean(axis=1)


# average of all prediction lengths
global_only_target_rmse = avg_only_target_rmse.mean()
global_with_other_tables_rmse = avg_with_other_tables_rmse.mean()
global_with_selected_features_rmse = avg_with_selected_features_rmse.mean()

In [ ]:
# summarize
rmses = [f'RMSE l={i}' for i in range(1,21)]
results = pd.DataFrame(columns = ['Time (s)', 'RMSE mean'] + rmses)

num_models = 24

results.loc['only target table'] = [only_target_time/5]+[global_only_target_rmse]+list(avg_only_target_rmse)
results.loc['with the features of other tables'] = [with_other_tables_time/5]+[global_with_other_tables_rmse]+list(avg_with_other_tables_rmse)
results.loc['with selected features of other tables'] = [with_selected_features_time/5]+[global_with_selected_features_rmse]+list(avg_with_selected_features_rmse)


results.values[:] = results.values.round(3)
results['Time (s)'] = results['Time (s)'].round(2)

results.to_csv('../docs/tables/selection_features_results.csv')
results

**Results**

In the situation when we leave the features of all correlated currencies, the model learning time increases by `7.7` times, and the quality of the model improves by `19%`!

After that we want to reduce the learning time of the model by additional selection by `feature_importance`. 
This selection allows us to reduce the training time by a factor of `10`, while losing only `8.3%` in accuracy.

Compared to the situation where we only use target currency data, we have an 

- `24%` speedup 
- `12%` increase in accuracy

# Conclusions

In [28]:
most_important_currencies = set()

for name in selected_features:
    right_id = name.find(') ')
    left_id = name.find('(')
    if right_id - left_id > 4:
        most_important_currencies.add(name[left_id + 1 : right_id])
        
# we already have "CHZ_USDT_PERP_MIDPRICE" currency
most_important_currencies.remove('CHZ_USDT_PERP')

In [29]:
from collections import defaultdict
import re

In [30]:
factorized_features = defaultdict(list)
for name in selected_features:
    for feature in most_important_currencies:       
        if feature in name:
            remove_feature_name = re.sub(f'\({feature}\)', '', name)
            remove_price_mean = re.sub(f'price_mean__', '', remove_feature_name)
            factorized_features[feature].append(remove_price_mean)

In [31]:
for key in factorized_features.keys():
    print(f'Features for "{key}" currency:')
    for el in factorized_features[key]:
        print('*  ', el)
    print('-' * 60)

Features for "CHZ_USDT_PERP_MIDPRICE" currency:
*    price_mean
*    mean_second_derivative_central
*    skewness
*    partial_autocorrelation__lag_3
*    cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)
*    fft_coefficient__attr_"imag"__coeff_70
*    friedrich_coefficients__coeff_3__m_3__r_30
------------------------------------------------------------
Features for "DOGE_USDT_PERP" currency:
*    skewness
*    cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)
------------------------------------------------------------
Features for "AVAX_USDT_PERP" currency:
*    skewness
*    mean_second_derivative_central
*    fft_coefficient__attr_"imag"__coeff_70
------------------------------------------------------------
Features for "ETH_BUSD_PERP" currency:
*    partial_autocorrelation__lag_3
*    price_std
------------------------------------------------------------
Features for "BTC_USDT_PERP" currency:
*    price_std
*    fft_coefficient__attr_"imag"__coeff_70
*    buy_price_sum
*

We see that for the currency series (`"DOGE_USDT_PERP"`, `"CHZ_USDT_PERP_MIDPRICE"`) it is useful to know the coefficients in the **Ricker's basis wavelet** decomposition. Unlike Fourier analysis, wavelet analysis is capable of capturing local aspects of the series. The coefficient numbers in the feature names should be treated with caution - all the importance of the features have been averaged by block cross validation. 

At the same time it is useful to have the **coefficients** of decomposition in a **Fourier** series (`"CHZ_USDT_PERP_MIDPRICE"`, `"AVAX_USDT_PERP"`, `"BTC_USDT_PERP"`)

A measure of the asymmetry of the value distribution also proved useful for currencies `"CHZ_USDT_PERP_MIDPRICE"`, `"DOGE_USDT_PERP"`, `"AVAX_USDT_PERP"`).

Finally, the **volume of purchases and sales** is important `"ETH_USDT_PERP"`, `"BTC_USDT_PERP"` and some other characteristics (**autocorrelation coefficients** and **variance value**)

For a detailed look at the complete solution scheme, you can read the **DOCS** and look at the `toy_example` notebook, which explains some functions on the toy example.